In [1]:
import utils
import predict
import time as tm
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.datasets import make_classification
import scipy
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data as data_utils
import torch.nn.functional as F

In [3]:
# load Data
dictSize = 225
(X, y) = utils.loadData( "train", dictSize = dictSize )
X = scipy.sparse.csr_matrix.toarray(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [4]:
# https://discuss.pytorch.org/t/multi-class-classification/47565
# https://medium.com/dair-ai/a-simple-neural-network-from-scratch-with-pytorch-and-google-colab-c7f3830618e0
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.layer2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.relu(out)
        out = self.layer2(out)
        return out
    
    def predict(self, x):
        with torch.no_grad():
            outp = self.forward(x)
            return F.softmax(outp)
# add softmax layer: used for multiclass classification

In [5]:
# model declared
model = NeuralNet(225 , 1000, 51)  
# earlier hidden layers- 1000 
# 1 - poor, 2- ok

In [6]:
loss_function = nn.CrossEntropyLoss()
model_opt = optim.SGD(model.parameters(), lr = 0.02)

train_set_X = Variable(torch.from_numpy(X_train)).float()
train_set_y = Variable(torch.LongTensor(y_train)).long()
test_set_X = Variable(torch.from_numpy(X_test)).float()
test_set_y = Variable(torch.LongTensor(y_test)).long()

In [7]:
# 1
epochs = 50
for epochs in range(epochs):
    model_opt.zero_grad()
    out = model(train_set_X)
    loss = loss_function(out, train_set_y)
    loss.backward()
    model_opt.step()

In [8]:
ans = model.predict(test_set_X)
# for i in range(3299):
#     ans_max,ind = torch.max(ans[i],0)
#     print (test_set_y[i].numpy() - ind.numpy())
    

/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [9]:
k = 5
y_pr = ans.numpy() 
y_pred = np.argsort(-y_pr,axis=1)[:,:k]
print (y_pred)
print (y_test)

[[3 2 4 1 7]
 [2 1 3 4 9]
 [2 3 1 4 7]
 ...
 [2 1 3 4 9]
 [2 1 4 3 9]
 [2 3 1 4 7]]
[3. 2. 3. ... 9. 4. 3.]


In [10]:
# eval functions for Deep Learning
preck = utils.getPrecAtK( y_test, y_pred, k )
# The macro precision code takes a bit longer to execute due to the for loop over labels
mpreck = utils.getMPrecAtK( y_test, y_pred, k )

# According to our definitions, both prec@k and mprec@k should go up as k goes up i.e. for your
# method, prec@i > prec@j if i > j and mprec@i > mprec@j if i > j. See the assignment description
# to convince yourself why this must be the case.

print( "prec@1: %0.3f" % preck[0], "prec@3: %0.3f" % preck[2], "prec@5: %0.3f" % preck[4] )
# Dont be surprised if mprec is small -- it is hard to do well on rare error classes
print( "mprec@1: %0.3e" % mpreck[0], "mprec@3: %0.3e" % mpreck[2], "mprec@5: %0.3e" % mpreck[4] )
print ("prec matrix",preck)
print ("mprec matrix",mpreck)

prec@1: 0.314 prec@3: 0.624 prec@5: 0.755
mprec@1: 2.944e-02 mprec@3: 7.486e-02 mprec@5: 1.340e-01
prec matrix [0.31363636 0.49878788 0.62363636 0.69363636 0.75515152]
mprec matrix [0.02943633 0.05661567 0.07486278 0.08510638 0.13399919]


In [11]:
# 2 
# same model diff train file

In [12]:
tr_latent_X = data_utils.TensorDataset(train_set_X, train_set_y)
te_latent_X = data_utils.TensorDataset(test_set_X,  test_set_y)
train_loader_X = torch.utils.data.DataLoader(dataset=tr_latent_X)
test_loader_X = torch.utils.data.DataLoader(dataset=te_latent_X)

In [13]:
def train(epoch):
    model.train()

    train_loss = 0

    for batch_idx, (data,label) in enumerate(train_loader_X):

        model_opt.zero_grad()

        out = model(data)
        loss = loss_function(out, label)

        loss.backward()
        train_loss += loss.item()
        model_opt.step()

        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader_X.dataset),
                100. * batch_idx / len(train_loader_X), loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch, train_loss / len(train_loader_X.dataset)))

In [14]:
# 2
for epoch in range(1, 10):
    train(epoch)

Train Epoch: 1 [0/6700 (0%)]	Loss: 2.895860
Train Epoch: 1 [100/6700 (1%)]	Loss: 2.147893
Train Epoch: 1 [200/6700 (3%)]	Loss: 0.775469
Train Epoch: 1 [300/6700 (4%)]	Loss: 4.259040
Train Epoch: 1 [400/6700 (6%)]	Loss: 3.802916
Train Epoch: 1 [500/6700 (7%)]	Loss: 1.437111
Train Epoch: 1 [600/6700 (9%)]	Loss: 1.776183
Train Epoch: 1 [700/6700 (10%)]	Loss: 0.854372
Train Epoch: 1 [800/6700 (12%)]	Loss: 2.214782
Train Epoch: 1 [900/6700 (13%)]	Loss: 0.539796
Train Epoch: 1 [1000/6700 (15%)]	Loss: 2.787828
Train Epoch: 1 [1100/6700 (16%)]	Loss: 0.022516
Train Epoch: 1 [1200/6700 (18%)]	Loss: 7.694416
Train Epoch: 1 [1300/6700 (19%)]	Loss: 0.167059
Train Epoch: 1 [1400/6700 (21%)]	Loss: 1.684062
Train Epoch: 1 [1500/6700 (22%)]	Loss: 0.702131
Train Epoch: 1 [1600/6700 (24%)]	Loss: 1.926263
Train Epoch: 1 [1700/6700 (25%)]	Loss: 0.049595
Train Epoch: 1 [1800/6700 (27%)]	Loss: 0.755877
Train Epoch: 1 [1900/6700 (28%)]	Loss: 1.157644
Train Epoch: 1 [2000/6700 (30%)]	Loss: 4.557193
Train Epoch

Train Epoch: 3 [3700/6700 (55%)]	Loss: 0.093891
Train Epoch: 3 [3800/6700 (57%)]	Loss: 0.056728
Train Epoch: 3 [3900/6700 (58%)]	Loss: 0.032450
Train Epoch: 3 [4000/6700 (60%)]	Loss: 2.632067
Train Epoch: 3 [4100/6700 (61%)]	Loss: 0.240265
Train Epoch: 3 [4200/6700 (63%)]	Loss: 0.009124
Train Epoch: 3 [4300/6700 (64%)]	Loss: 1.147588
Train Epoch: 3 [4400/6700 (66%)]	Loss: 0.001979
Train Epoch: 3 [4500/6700 (67%)]	Loss: 1.951003
Train Epoch: 3 [4600/6700 (69%)]	Loss: 2.078163
Train Epoch: 3 [4700/6700 (70%)]	Loss: 0.197971
Train Epoch: 3 [4800/6700 (72%)]	Loss: 2.389737
Train Epoch: 3 [4900/6700 (73%)]	Loss: 0.310630
Train Epoch: 3 [5000/6700 (75%)]	Loss: 1.668003
Train Epoch: 3 [5100/6700 (76%)]	Loss: 0.100045
Train Epoch: 3 [5200/6700 (78%)]	Loss: 1.417741
Train Epoch: 3 [5300/6700 (79%)]	Loss: 0.544871
Train Epoch: 3 [5400/6700 (81%)]	Loss: 0.141199
Train Epoch: 3 [5500/6700 (82%)]	Loss: 2.359682
Train Epoch: 3 [5600/6700 (84%)]	Loss: 2.878024
Train Epoch: 3 [5700/6700 (85%)]	Loss: 2

Train Epoch: 6 [700/6700 (10%)]	Loss: 0.413284
Train Epoch: 6 [800/6700 (12%)]	Loss: 0.959520
Train Epoch: 6 [900/6700 (13%)]	Loss: 0.073325
Train Epoch: 6 [1000/6700 (15%)]	Loss: 1.739574
Train Epoch: 6 [1100/6700 (16%)]	Loss: 0.000003
Train Epoch: 6 [1200/6700 (18%)]	Loss: 3.785931
Train Epoch: 6 [1300/6700 (19%)]	Loss: 0.011897
Train Epoch: 6 [1400/6700 (21%)]	Loss: 0.020853
Train Epoch: 6 [1500/6700 (22%)]	Loss: 0.159387
Train Epoch: 6 [1600/6700 (24%)]	Loss: 0.011934
Train Epoch: 6 [1700/6700 (25%)]	Loss: 0.001540
Train Epoch: 6 [1800/6700 (27%)]	Loss: 0.034458
Train Epoch: 6 [1900/6700 (28%)]	Loss: 0.427896
Train Epoch: 6 [2000/6700 (30%)]	Loss: 6.960698
Train Epoch: 6 [2100/6700 (31%)]	Loss: 0.025643
Train Epoch: 6 [2200/6700 (33%)]	Loss: 0.666192
Train Epoch: 6 [2300/6700 (34%)]	Loss: 0.007638
Train Epoch: 6 [2400/6700 (36%)]	Loss: 0.067081
Train Epoch: 6 [2500/6700 (37%)]	Loss: 0.044136
Train Epoch: 6 [2600/6700 (39%)]	Loss: 0.244827
Train Epoch: 6 [2700/6700 (40%)]	Loss: 0.48

Train Epoch: 8 [4400/6700 (66%)]	Loss: 0.000009
Train Epoch: 8 [4500/6700 (67%)]	Loss: 0.855413
Train Epoch: 8 [4600/6700 (69%)]	Loss: 0.878460
Train Epoch: 8 [4700/6700 (70%)]	Loss: 0.020851
Train Epoch: 8 [4800/6700 (72%)]	Loss: 2.986957
Train Epoch: 8 [4900/6700 (73%)]	Loss: 0.011705
Train Epoch: 8 [5000/6700 (75%)]	Loss: 0.390906
Train Epoch: 8 [5100/6700 (76%)]	Loss: 0.015311
Train Epoch: 8 [5200/6700 (78%)]	Loss: 0.286307
Train Epoch: 8 [5300/6700 (79%)]	Loss: 0.268280
Train Epoch: 8 [5400/6700 (81%)]	Loss: 0.045730
Train Epoch: 8 [5500/6700 (82%)]	Loss: 0.786717
Train Epoch: 8 [5600/6700 (84%)]	Loss: 0.926244
Train Epoch: 8 [5700/6700 (85%)]	Loss: 3.377732
Train Epoch: 8 [5800/6700 (87%)]	Loss: 1.075761
Train Epoch: 8 [5900/6700 (88%)]	Loss: 0.195868
Train Epoch: 8 [6000/6700 (90%)]	Loss: 0.491252
Train Epoch: 8 [6100/6700 (91%)]	Loss: 0.000632
Train Epoch: 8 [6200/6700 (93%)]	Loss: 0.010817
Train Epoch: 8 [6300/6700 (94%)]	Loss: 0.010576
Train Epoch: 8 [6400/6700 (96%)]	Loss: 0

In [15]:
ans = model.predict(test_set_X)
k = 5
y_pr = ans.numpy() 
y_pred = np.argsort(-y_pr,axis=1)[:,:k]
print (y_pred)
print (y_test)

[[ 3 12 22  7  4]
 [ 2  1  9  4 15]
 [ 3 12  4 39 22]
 ...
 [ 9  2 16 45  1]
 [ 4 37  5 16 18]
 [ 3 12 22 37 10]]
[3. 2. 3. ... 9. 4. 3.]


/home/aditya/Vpy35/lib/python3.5/site-packages/ipykernel_launcher.py:19: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [16]:
# eval functions for Deep Learning
preck = utils.getPrecAtK( y_test, y_pred, k )
# The macro precision code takes a bit longer to execute due to the for loop over labels
mpreck = utils.getMPrecAtK( y_test, y_pred, k )

# According to our definitions, both prec@k and mprec@k should go up as k goes up i.e. for your
# method, prec@i > prec@j if i > j and mprec@i > mprec@j if i > j. See the assignment description
# to convince yourself why this must be the case.

print( "prec@1: %0.3f" % preck[0], "prec@3: %0.3f" % preck[2], "prec@5: %0.3f" % preck[4] )
# Dont be surprised if mprec is small -- it is hard to do well on rare error classes
print( "mprec@1: %0.3e" % mpreck[0], "mprec@3: %0.3e" % mpreck[2], "mprec@5: %0.3e" % mpreck[4] )
print ("prec matrix",preck)
print ("mprec matrix",mpreck)

prec@1: 0.774 prec@3: 0.917 prec@5: 0.950
mprec@1: 5.034e-01 mprec@3: 7.645e-01 mprec@5: 8.343e-01
prec matrix [0.77363636 0.87909091 0.9169697  0.94060606 0.94969697]
mprec matrix [0.50340555 0.69311973 0.7645074  0.81593861 0.83426069]
